In [1]:
using Distributions, Turing
using Gadfly

In [2]:
@model gdemo2(x, bkstep) = begin
    y = similar(x)
    if bkstep == false  
        # Forward Sample
        s ~ InverseGamma(2,3)
        m ~ Normal(0,sqrt(s))
        y[1] ~ Normal(m, sqrt(s))
        y[2] ~ Normal(m, sqrt(s))
    elseif bkstep == true 
        # Backward Step 1: theta ~ theta | x
        s ~ InverseGamma(2,3)
        m ~ Normal(0,sqrt(s))
        x[1] ~ Normal(m, sqrt(s))
        x[2] ~ Normal(m, sqrt(s))
        # Backward Step 2: x ~ x | theta
        y[1] ~ Normal(m, sqrt(s))
        y[2] ~ Normal(m, sqrt(s))
    end
    return s, m, y
end

 Assume - `s` is a parameter
 Assume - `m` is a parameter
 Assume - `y` is a parameter
 Observe - `x` is an observation


gdemo2 (generic function with 4 methods)

In [3]:
fw = PG(20, 300)
bk = Gibbs(10, PG(10,10, :s, :y), HMC(1, 0.25, 5, :m));

In [4]:
s = @sample(gdemo2([1.5, 2], false), fw);

[PG]: Finshed within 2.7866458892822266 seconds


In [5]:
describe(s)

Iterations = 1:300
Thinning interval = 1
Chains = 1
Samples per chain = 300

Empirical Posterior Estimates:
         Mean        SD     Naive SE      MCSE    ESS
   m 0.186876338 1.5828736 0.09138725 0.031917705 300
y[1] 0.021829037 2.4501294 0.14145829 0.080212894 300
y[2] 0.295076622 2.4325274 0.14044203 0.103072231 300
   s 3.253114980 5.5768925 0.32198204 0.073906481 300

Quantiles:
         2.5%       25.0%       50.0%      75.0%      97.5%  
   m -2.74503914 -0.76699002 0.118378945 1.0859695  3.4705196
y[1] -4.06774709 -1.35406974 0.044656838 1.2878395  4.0238171
y[2] -3.71981601 -0.95970933 0.145579527 1.4402614  5.3891412
   s  0.48593145  1.08023792 1.781509454 3.3918546 13.4523979



In [ ]:
N = 100
x = [s[:y][1]...]
s_bk = Array{Turing.Chain}(N)

for i = 1:N
    s_bk[i] = @sample(gdemo2(x, true), bk);
    x = [s_bk[i][:y][1]...];
end

[Gibbs]: Finshed within 4.269287109375 seconds
[Gibbs]: Finshed within 0.07929396629333496 seconds
[Gibbs]: Finshed within 0.0547490119934082 seconds
[Gibbs]: Finshed within 0.06913399696350098 seconds
[Gibbs]: Finshed within 0.06143689155578613 seconds
[Gibbs]: Finshed within 0.09006118774414062 seconds


In [ ]:
s2 = vcat(s_bk...);
describe(s2)

In [ ]:
p = plot(s);

In [ ]:
hstack(p[1:2])

In [ ]:
hstack(p[3:4])

In [ ]:
heideldiag(s)

In [ ]:
import Turing.@sample

macro sample(modelcall, alg, optionalps...)
  modelf = modelcall.args[1]      # get the function symbol for model
  psyms = modelcall.args[2:end]   # get the (passed-in) symbols
  esc(quote
    data_ = Dict()
    arglist_ = Turing.TURING[:modelarglist]
    localsyms_ = $psyms
    for i_ = 1:length(arglist_)
      localsym_ = localsyms_[i_]     # passed-in symbols are local
      arg_ = arglist_[i_]            # arglist are symbols in model scope
      data_[arg_] = eval(localsym_)
    end
    sample($modelf, data_, $alg, $optionalps...)
  end)
end